In [ ]:
#hide
%load_ext autoreload
%autoreload 2

In [ ]:
#hide
from gate_simulation.macrotools import *

# Simulation of the proton beam tracker/calorimeter

> This project will have tools to aid a simulation and optimizatin of the proton beam tracker/calorimeter in application for determining the corrections to the proton stopping powers for treatment plannig.

## Prerequisites

To use this package you need to first install:
* [root](https://root.cern/install/) 
* [Gate](http://www.opengatecollaboration.org)
    * I am using a version of Gate I cloned from [their Github](https://github.com/OpenGATE/Gate) page, because the official 9.0 verion has a bug.
* [Geant4](https://geant4.web.cern.ch/support/download)
    * Check the requirements for the correct verion of Geant4 on [OpenGate](http://www.opengatecollaboration.org) page
* [Cmake](https://cmake.org/download/)

## Install

**Not working yet**

~~`pip install gate_simulation`~~

## How to use

Create a sensor:

In [ ]:
create_sensor(n=2)

Recalculating the kinetic energy $E_k$ of the  particle mass  $M$ from its momentum $p$  according to:
$$E_k = \sqrt{M^2  + p^2} - M$$

In [ ]:
print(f'The kinetic energy of 2 GeV/c proton is {Ek(938,2000):.0f} MeV')

The kinetic energy of 2 GeV/c proton is 1271 MeV


In [ ]:
#hide
from nbdev.export import notebook2script; notebook2script()

Converted 00_core.ipynb.
Converted index.ipynb.


In [ ]:
!ls mac

my_proton_beam.mac sensor1.mac        visu.mac
sensor0.mac        sensor2.mac
